[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeffneo/call-transcripts-automation/blob/main/students/3_Cleanup.ipynb)

In [1]:
%%capture
!pip --quiet install python-dotenv neo4j graphdatascience
!pip --quiet install langchain_neo4j langchain_openai langgraph

## Dotenv configuration

In [2]:
# NOTE: Upload files (.env)!!!!
# You can skip this cell if not using a ws.env file - alternative to above
from dotenv import load_dotenv
import os

if os.path.exists('aura.txt'):
     load_dotenv('aura.txt', override=True)

     # Neo4j
     NEO4J_URI = os.getenv('NEO4J_URI')
     NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
     NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

     AURA_API_CLIENT_ID=os.getenv('AURA_API_CLIENT_ID')
     AURA_API_CLIENT_SECRET=os.getenv('AURA_API_CLIENT_SECRET')

     IS_AURA = os.environ.get("AURA", True) is True

     # AI
     LLM = 'gpt-4o'
     AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

print(IS_AURA)
print(AZURE_OPENAI_API_KEY)

True
6CgahPtoFmvzqh2gsBzef795O95YTMrIHg4KqNjYfOjvbC3BTdXmJQQJ99BCACYeBjFXJ3w3AAABACOGtZuK


## Connecting to Neo4j

Provide your Neo4j credentials. We need the DB conection URL, the username (probably neo4j), and your password.

In [3]:
# # username is neo4j by default
# NEO4J_USERNAME = 'neo4j'

# # You will need to change these to match your credentials
# NEO4J_URI = 'neo4j+s://a51c4bb3.databases.neo4j.io'
# NEO4J_PASSWORD = 'R50r64jHpBzFgxZS6tttK7MXwO5agK6syL-Pznw5m_Y'

In [8]:
from langchain_neo4j import Neo4jGraph

# You can skip this cell if not using a ws.env file - alternative to above
# Get credentials from .env file
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

graph_db = Neo4jGraph(url=os.getenv("NEO4J_URI"), username=os.getenv("NEO4J_USERNAME"), password=os.getenv("NEO4J_PASSWORD"))

In [5]:
# from graphdatascience import GraphDataScience

# gds = GraphDataScience(
#     NEO4J_URI,
#     auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
#     aura_ds=False
# )
# gds.set_database('neo4j')

# Cleanup Post Ingestion

In [13]:
graph_db.query("""
  MATCH (n:Observation|ProcessElement)
  CALL (n) {
    DETACH DELETE n
  } IN TRANSACTIONS OF 1000 ROWS
  return count(n) as deleted
  """)

[{'deleted': 0}]

# Cleanup everything

In [6]:
graph_db.query("""
  MATCH (n)
  CALL (n) {
    DETACH DELETE n
  } IN TRANSACTIONS OF 1000 ROWS
  return count(n) as deleted
  """)

[]